In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import nuwave2
from malaya_speech.train.model.nuwave2 import model as nuwave2_model
from malaya_boilerplate.train import config
from malaya_speech.train.model.nuwave2 import prepare_dataset
import numpy as np

In [5]:
hparams = config.HParams(**malaya_speech.config.nuwave2_config)
hparams

{'train': {'batch_size': 12, 'lr': '2e-4', 'weight_decay': 0.0, 'num_workers': 8, 'gpus': 2, 'opt_eps': '1e-9', 'beta1': 0.9, 'beta2': 0.99}, 'data': {'timestamp_path': 'vctk-silence-labels/vctk-silences.0.92.txt', 'base_dir': '/DATA1/VCTK-0.92/wav48_silence_trimmed/', 'dir': '/DATA1/VCTK-0.92/wav48_silence_trimmed_wav/', 'format': '*mic1.wav', 'cv_ratio': '(100./108., 8./108., 0.00)'}, 'audio': {'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'sampling_rate': 48000, 'sr_min': 6000, 'sr_max': 48000, 'length': 32768}, 'arch': {'residual_layers': 15, 'residual_channels': 64, 'pos_emb_dim': 512, 'bsft_channels': 64}, 'logsnr': {'logsnr_min': -20.0, 'logsnr_max': 20.0}, 'dpm': {'max_step': 1000, 'pos_emb_scale': 50000, 'pos_emb_channels': 128, 'infer_step': 8, 'infer_schedule': 'tf.constant([-2.6, -0.8, 2.0, 6.4, 9.8, 12.9, 14.4, 17.2])'}, 'log': {'name': 'nuwave2', 'checkpoint_dir': 'checkpoint', 'tensorboard_dir': 'tensorboard', 'test_result_dir': 'test_sample/result'}}

In [6]:
from glob import glob

audios = glob('../speech/44k/*.wav')
audios

['../speech/44k/test-3.wav',
 '../speech/44k/test-2.wav',
 '../speech/44k/test-0.wav',
 '../speech/44k/test-1.wav']

In [7]:
hparams.audio.sampling_rate

48000

In [8]:
dataset = prepare_dataset.Dataset(audios, hparams)

In [9]:
wav, wav_l, band = dataset[0]
wav.shape, wav_l.shape

((32768,), (32768,))

In [10]:
model = nuwave2.Model(hparams)

In [11]:
wavs, wav_ls, bands, t = dataset.batch([wav, wav], [wav_l, wav_l], [band, band])

In [12]:
noise_estimation, z, logsnr, wav, wav_noisy, logsnr = model(wavs, wav_ls, bands, t.astype(np.float32))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the ver

In [13]:
mae = tf.losses.absolute_difference

In [14]:
mae_loss = mae(
    labels=z, predictions=noise_estimation
)

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-4, beta1 = 0.9, beta2 = 0.99, epsilon = 1e-9).minimize(mae_loss)
optimizer

<tf.Operation 'Adam' type=NoOp>

In [16]:
l_audios = glob('../speech/example-speaker/*.wav')
l_audios

['../speech/example-speaker/haqkiem.wav',
 '../speech/example-speaker/husein-generated.wav',
 '../speech/example-speaker/khalil-nooh.wav',
 '../speech/example-speaker/muhyiddin-yassin.wav',
 '../speech/example-speaker/mas-aisyah.wav',
 '../speech/example-speaker/female.wav',
 '../speech/example-speaker/shafiqah-idayu.wav',
 '../speech/example-speaker/husein-zolkepli.wav']

In [17]:
y, sr = malaya_speech.load(l_audios[0], sr = None)
y = y[:sr]
sr, len(y) / sr

(22050, 1.0)

In [18]:
from scipy.signal import resample_poly
y_l = resample_poly(y, hparams.audio.sampling_rate, sr)
y_l = y_l[:len(y_l) - len(y_l) % hparams.audio.hop_length]
len(y_l)

47872

In [19]:
highcut = sr // 2
nyq = 0.5 * hparams.audio.sampling_rate
hi = highcut / nyq

fft_size = hparams.audio.filter_length // 2 + 1
band = np.zeros(fft_size, dtype=np.int32)
band[:int(hi * fft_size)] = 1

In [20]:
steps = 8
noise_schedule = None
wav_recon, wav_list = model.inference(np.expand_dims(y_l, 0), np.expand_dims(band, 0), steps, noise_schedule)

In [21]:
wav_recon

<tf.Tensor 'clip_by_value:0' shape=(1, 47872) dtype=float32>

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
sess.run([mae_loss, optimizer])

[0.8528071, None]

In [24]:
wav_, wav_noisy_ = sess.run([wav, wav_noisy])

In [25]:
wav_.shape, wav_.max(), wav_.min()

((2, 32768), 0.59448874, -0.4020105)

In [26]:
wav_noisy_.max(), wav_noisy_.min()

(3.468558, -3.3615136)

In [27]:
y_ = sess.run(wav_recon)
y_.shape

(1, 47872)

In [ ]:
# for i in range(20):
#     print(i, sess.run([mae_loss, optimizer]))